In [ ]:
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

# Load OpenAI API key from .env file
TOKEN = os.getenv('TOKEN')

client = OpenAI(api_key=os.environ.get('TOKEN'))

# Load housing data from CSV
housing_data = pd.read_csv('Housing.csv')

# Sends a request to the OpenAI GPT-3.5-turbo model to generate content.
def make_openai_request(prompts, n=1):
    try:
        # Preparing messages for OpenAI API request
        messages = [{"role": "system", "content": prompt} for prompt in prompts] + [{"role": "user", "content": ""}]
        # Sending a request to the OpenAI API to generate content
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=150,
            n=n,
            stop=None,
            temperature=0.7
        )
        return [choice.message.content for choice in response.choices]
    except Exception as e:
        print(f"Error making OpenAI API request: {e}")
        return ["Failed to generate post. Please check your input data."]
    
# Generates content for a real estate listing using OpenAI GPT-3.5-turbo.
def generate_posts(entries, n=1):
    prompts = [f"Create an engaging Instagram post for this real estate listing:\n\n{entry}\n\nCaption: " for entry in entries]
    return make_openai_request(prompts, n)

#The main function for organizing the process of generating content and saving the results in a file.
def main():
    examples = []
    # Sample 20 different dataset entries without replacement
    dataset_entries = housing_data.sample(n=20, replace=False).to_dict('records')
    # Generate content for each property listing
    post_texts = generate_posts(dataset_entries, n=20)

    # Save examples to the list
    for entry, post_text in zip(dataset_entries, post_texts):
        examples.append(f"price={entry['price']}\narea={entry['area']}\n{post_text}\n------------------------")
            
    # Save examples to EXAMPLE.txt
    with open("EXAMPLE.txt", "w", encoding="utf-8") as example_file:
        example_file.write("\n".join(examples))

if __name__ == "__main__":
    main()
